In [ ]:
import pandas as pd

df_both = pd.read_csv('/content/Dataset Both.csv')
df_vivo = pd.read_csv('/content/dataset in vivo.csv')
df_vitro = pd.read_csv('/content/Dataset in vitro.csv')

# Quantidade de dados por dataset

In [ ]:
df_vivo = df_vivo.drop(columns=['Type', 'Species','Strain', 'Male', 'Female'])
df_vitro["Results"] = df_vitro['In vitro Result']

In [ ]:
df_both = df_both.drop(columns=['PUBCHEM_CID','Chemical' ])

In [ ]:
df_vivo = df_vivo.drop_duplicates()
df_vitro = df_vitro.drop_duplicates()
df_both = df_both.drop_duplicates()

In [ ]:
print(f'Dados no dataset Vivo:', df_vivo.shape)
print(f'Dados no dataset Vitro:', df_vitro.shape)
print(f'Dados no dataset Both:', df_both.shape)

Dados no dataset Vivo: (4966, 4)
Dados no dataset Vitro: (3325, 5)
Dados no dataset Both: (7414, 3)


# Quantidade de moleculas distintas

In [ ]:
print(f'moleculas unicas no dataset vivo:', df_vivo['SMILES'].nunique())
print(f'moleculas unicas no dataset vitro:', df_vitro['SMILES'].nunique())
print(f'moleculas unicas no dataset both:', df_both['SMILES'].nunique())

moleculas unicas no dataset vivo: 2917
moleculas unicas no dataset vitro: 2980
moleculas unicas no dataset both: 4111


# Resultados positivos e negativos por dataset

In [ ]:
# Contar os valores únicos na coluna 'Results' para cada DataFrame
df_both["Results"] = df_both["Results"].str.lower()
df_vitro["Results"] = df_vitro["Results"].str.lower()
df_vivo["Results"] = df_vivo["Results"].str.lower()

# Contar os valores únicos na coluna 'Results' para cada DataFrame
result_counts = {
    "df_vivo": df_vivo["Results"].value_counts(),
    "df_vitro": df_vitro["Results"].value_counts(),
    "df_both": df_both["Results"].value_counts(),
}

# Criar um DataFrame com os resultados
result_df = pd.DataFrame(result_counts).fillna(0).astype(int)

result_df

,df_vivo,df_vitro,df_both
Results,,,
ambiguous,0,0,8
inconclusive,0,0,4
negative,2691,1320,3220
positive,2275,2005,3920


In [ ]:
# Remover linhas onde "Results" é "ambiguous" ou "inconclusive"
df_both = df_both[~df_both["Results"].isin(["ambiguous", "inconclusive"])]

# Resetar o índice para manter o DataFrame organizado
df_both = df_both.reset_index(drop=True)

# Quantidade de moleculas com mais de um resultado

In [ ]:
df_both["Results"] = df_both["Results"].str.lower()

smiles_result_counts = df_both.groupby("SMILES")["Results"].nunique()

count_one_results = (smiles_result_counts == 1).sum()
count_2_results = (smiles_result_counts == 2).sum()
count_3_results = (smiles_result_counts > 2).sum()

smiles_result_comparison_df = pd.DataFrame({
    "Tipo de Contagem": ["Apenas um resultado", "Dois resultados", "Mais de dois resultados"],
    "Quantidade": [count_one_results, count_2_results, count_3_results]
})

print(f'moleculas unicas no dataset both:', df_both['SMILES'].nunique())

smiles_result_comparison_df

moleculas unicas no dataset both: 4111


,Tipo de Contagem,Quantidade
0,Apenas um resultado,3062
1,Dois resultados,1049
2,Mais de dois resultados,0


# Quantidade de moleculas com dados in vivo e in vitro.

In [ ]:
# Contar a quantidade total de registros para "in vivo" e "in vitro" no df_both
num_total_in_vivo = (df_both["Type"] == "in vivo").sum()
num_total_in_vitro = (df_both["Type"] == "in vitro").sum()

# Criar um DataFrame com os valores contados
df_total_summary = pd.DataFrame({
    "Type": ["in vivo", "in vitro"],
    "Quantidade Total": [num_total_in_vivo, num_total_in_vitro]
})
df_total_summary

,Type,Quantidade Total
0,in vivo,4269
1,in vitro,3133


# Quantidade de divergencia de results em msm type

In [ ]:
df_both["Type"] = df_both["Type"].str.lower()

smiles_type_counts = df_both.groupby("SMILES")["Type"].nunique()

count_one_type = (smiles_type_counts == 1).sum()
count_multiple_types = (smiles_type_counts > 1).sum()

smiles_type_comparison_df = pd.DataFrame({
    "Tipo de Contagem": ["Apenas um Type", "Mais de um Type"],
    "Quantidade": [count_one_type, count_multiple_types]
})

smiles_type_comparison_df

,Tipo de Contagem,Quantidade
0,Apenas um Type,2803
1,Mais de um Type,1308


# Quantidade De Positivo por type, em dados divergentes.

In [ ]:
# Contar quantos tipos diferentes de "Results" cada combinação de "SMILES" e "Type" tem
smiles_type_results_counts = df_both.groupby(["SMILES", "Type"])["Results"].nunique()

# Selecionar apenas os casos onde há mais de um resultado diferente
smiles_with_multiple_results = smiles_type_results_counts[smiles_type_results_counts > 1].index
smiles_without_multiple_results = smiles_type_results_counts[smiles_type_results_counts == 1].index

df_no_multiple_results = df_both.set_index(["SMILES", "Type"]).loc[smiles_without_multiple_results].reset_index()

#selecionar apenas onde são divergentes por type
multiple_results = df_no_multiple_results.groupby("SMILES")["Results"].nunique()

# Selecionar apenas o result dos types são divergentes
smiles_with_multiple_results = smiles_type_results_counts[smiles_type_results_counts > 1].index

# Criar um DataFrame apenas com esses SMILES e Types
df_multiple_results = df_both.set_index(["SMILES", "Type"]).loc[smiles_with_multiple_results].reset_index()

# Filtrar apenas os registros onde "Results" é "positive"
df_positive_results = df_multiple_results[df_multiple_results["Results"] == "positive"]
num_in_vivo = (df_positive_results["Type"] == "in vivo").sum()
num_in_vitro = (df_positive_results["Type"] == "in vitro").sum()

# Criar um DataFrame com os valores contados
df_summary = pd.DataFrame({
    "Type": ["in vivo", "in vitro"],
    "Quantidade": [num_in_vivo, num_in_vitro]
})

df_summary

,Type,Quantidade
0,in vivo,821
1,in vitro,404


# Teste estatistico

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
# Definir as contagens de positivos e totais para cada grupo
total = num_in_vivo + num_in_vitro
positive_counts = [int(num_in_vivo ), int(num_in_vitro)]
total_counts = [total, total]

# Realizar o teste Z para duas proporções
stat, p_value = proportions_ztest(positive_counts, total_counts)

# Criar DataFrame para exibição dos resultados
test_results_df = pd.DataFrame({
    "Estatística Z": [stat],
    "Valor-p": [p_value]
})

# Definir o nível de significância
alpha = 0.05

print(f'% total in vivo:' , (num_in_vivo/total) * 100)
# Analisar o resultado do teste
if p_value < alpha:
    conclusion = "Rejeitamos a hipótese nula. Há uma diferença estatisticamente significativa entre os grupos in vitro e in vivo."
else:
    conclusion = "Não rejeitamos a hipótese nula. Não há evidência estatística suficiente para afirmar que há diferença entre os grupos in vitro e in vivo."

# Adicionar a conclusão ao DataFrame
print(conclusion)

% total in vivo: 67.02040816326532
Rejeitamos a hipótese nula. Há uma diferença estatisticamente significativa entre os grupos in vitro e in vivo.
